In [1]:
import time
import pandas as pd
import vectorbt as vbt
import logging
from binance import enums

In [2]:
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
# Binance
vbt.settings.data['binance']['api_key'] =  "YOUR_API_KEY"
vbt.settings.data['binance']['api_secret'] = "YOUR_API_SECRET"

# Telegram
vbt.settings.messaging['telegram']['token'] = "YOUR_TOKEN"

# Giphy
vbt.settings.messaging['giphy']['api_key'] = "YOUR_API_KEY"

# Data
SYMBOLS = ['BTCUSDT', 'ETHUSDT', 'ADAUSDT']
START = '1 hour ago UTC'
INTERVAL = enums.KLINE_INTERVAL_1MINUTE
UPDATE_EVERY = vbt.utils.datetime.interval_to_ms(INTERVAL) // 1000  # in seconds
TZ_CONVERT = vbt.utils.datetime.get_utc_tz()
DT_FORMAT = '%d %b %Y %H:%M:%S %z'
IND_PARAMS = dict(
    timeperiod=20, 
    nbdevup=2, 
    nbdevdn=2
)
CHANGE_NBDEV = 2

In [4]:
data = vbt.BinanceData.download(
    SYMBOLS, 
    start=START, 
    end='now UTC', 
    interval=INTERVAL, 
    tz_convert=TZ_CONVERT
)

print(data.wrapper.index)

DatetimeIndex(['2021-03-22 13:21:00+00:00', '2021-03-22 13:22:00+00:00',
               '2021-03-22 13:23:00+00:00', '2021-03-22 13:24:00+00:00',
               '2021-03-22 13:25:00+00:00', '2021-03-22 13:26:00+00:00',
               '2021-03-22 13:27:00+00:00', '2021-03-22 13:28:00+00:00',
               '2021-03-22 13:29:00+00:00', '2021-03-22 13:30:00+00:00',
               '2021-03-22 13:31:00+00:00', '2021-03-22 13:32:00+00:00',
               '2021-03-22 13:33:00+00:00', '2021-03-22 13:34:00+00:00',
               '2021-03-22 13:35:00+00:00', '2021-03-22 13:36:00+00:00',
               '2021-03-22 13:37:00+00:00', '2021-03-22 13:38:00+00:00',
               '2021-03-22 13:39:00+00:00', '2021-03-22 13:40:00+00:00',
               '2021-03-22 13:41:00+00:00', '2021-03-22 13:42:00+00:00',
               '2021-03-22 13:43:00+00:00', '2021-03-22 13:44:00+00:00',
               '2021-03-22 13:45:00+00:00', '2021-03-22 13:46:00+00:00',
               '2021-03-22 13:47:00+00:00', '2021-0

In [5]:
def get_bbands(data):
    return vbt.IndicatorFactory.from_talib('BBANDS').run(
        data.get('Close'), **IND_PARAMS, hide_params=list(IND_PARAMS.keys()))


def get_info(bbands):
    info = dict()
    info['last_price'] = bbands.close.iloc[-1]
    info['last_change'] = (bbands.close.iloc[-1] - bbands.close.iloc[-2]) / bbands.close.iloc[-1]
    info['last_crossed_above_upper'] = bbands.close_above(bbands.upperband, crossover=True).iloc[-1]
    info['last_crossed_below_upper'] = bbands.close_below(bbands.upperband, crossover=True).iloc[-1]
    info['last_crossed_below_lower'] = bbands.close_below(bbands.lowerband, crossover=True).iloc[-1]
    info['last_crossed_above_lower'] = bbands.close_above(bbands.lowerband, crossover=True).iloc[-1]
    info['bw'] = (bbands.upperband - bbands.lowerband) / bbands.middleband
    info['last_bw_zscore'] = info['bw'].vbt.zscore().iloc[-1]
    info['last_change_zscore'] = bbands.close.vbt.pct_change().vbt.zscore().iloc[-1]
    info['last_change_pos'] = info['last_change_zscore'] >= CHANGE_NBDEV
    info['last_change_neg'] = info['last_change_zscore'] <= -CHANGE_NBDEV
    return info


def format_symbol_info(symbol, info):
    last_change = info['last_change'][symbol]
    last_price = info['last_price'][symbol]
    last_bw_zscore = info['last_bw_zscore'][symbol]
    return "{} ({:.2%}, {}, {:.2f})".format(symbol, last_change, last_price, last_bw_zscore)


def format_signals_info(emoji, signals, info):
    symbols = signals.index[signals]
    symbol_msgs = []
    for symbol in symbols:
        symbol_msgs.append(format_symbol_info(symbol, info))
    return "{} {}".format(emoji, ', '.join(symbol_msgs))

In [6]:
from telegram.ext import CommandHandler

class MyTelegramBot(vbt.TelegramBot):
    def __init__(self, data, *args, **kwargs):
        super().__init__(data=data, *args, **kwargs)
        
        self.data = data
        self.update_ts = data.wrapper.index[-1]
        
    @property
    def custom_handlers(self):
        return (CommandHandler('info', self.info_callback),)
    
    def info_callback(self, update, context):
        chat_id = update.effective_chat.id
        if len(context.args) != 1:
            self.send_message(chat_id, "Please provide one symbol.")
            return
        symbol = context.args[0]
        if symbol not in SYMBOLS:
            self.send_message(chat_id, f"There is no such symbol as \"{symbol}\".")
            return
            
        bbands = get_bbands(self.data)
        info = get_info(bbands)
        messages = [format_symbol_info(symbol, info)]
        message = '\n'.join(["{}:".format(self.update_ts.strftime(DT_FORMAT))] + messages)
        self.send_message(chat_id, message)
        
    @property
    def start_message(self):
        index = self.data.wrapper.index
        return f"""Hello! 

Starting with {len(index)} rows from {index[0].strftime(DT_FORMAT)} to {index[-1].strftime(DT_FORMAT)}."""
        
    @property
    def help_message(self):
        return """Message format:
[event] [symbol] ([price change], [new price], [bandwidth z-score])
    
Event legend:
⬆️ - Price went above upper band
⤵️ - Price retraced below upper band
⬇️ - Price went below lower band
⤴️ - Price retraced above lower band

GIF is sent once a band is crossed and the price change is 2 stds from the mean."""

In [7]:
telegram_bot = MyTelegramBot(data)
telegram_bot.start(in_background=True)

2021-03-22 15:20:14,468 - vectorbt.utils.messaging - INFO - Initializing bot
2021-03-22 15:20:14,487 - vectorbt.utils.messaging - INFO - Loaded chat ids [447924619]
2021-03-22 15:20:14,591 - vectorbt.utils.messaging - INFO - Running bot vectorbt_bot
2021-03-22 15:20:14,592 - apscheduler.scheduler - INFO - Scheduler started


In [8]:
class MyDataUpdater(vbt.DataUpdater):
    def __init__(self, data, telegram_bot, **kwargs):
        super().__init__(data, telegram_bot=telegram_bot, **kwargs)
        
        self.telegram_bot = telegram_bot
        self.update_ts = data.wrapper.index[-1]
        
    def update(self):
        super().update()
        self.update_ts = pd.Timestamp.now(tz=TZ_CONVERT)
        self.telegram_bot.data = self.data
        self.telegram_bot.update_ts = self.update_ts
        
        bbands = get_bbands(self.data)
        info = get_info(bbands)
        
        messages = []
        if info['last_crossed_above_upper'].any():
            messages.append(format_signals_info('⬆️', info['last_crossed_above_upper'], info))
        if info['last_crossed_below_upper'].any():
            messages.append(format_signals_info('⤵️', info['last_crossed_below_upper'], info))
        if info['last_crossed_below_lower'].any():
            messages.append(format_signals_info('⬇️', info['last_crossed_below_lower'], info))
        if info['last_crossed_above_lower'].any():
            messages.append(format_signals_info('⤴️', info['last_crossed_above_lower'], info))
            
        if len(messages) > 0:
            message = '\n'.join(["{}:".format(self.update_ts.strftime(DT_FORMAT))] + messages)
            self.telegram_bot.send_message_to_all(message)
        if (info['last_crossed_above_upper'] & info['last_change_pos']).any():
            self.telegram_bot.send_giphy_to_all("launch")
        if (info['last_crossed_below_lower'] & info['last_change_neg']).any():
            self.telegram_bot.send_giphy_to_all("fall")

In [9]:
data_updater = MyDataUpdater(data, telegram_bot)
data_updater.update_every(UPDATE_EVERY)

2021-03-22 15:20:14,756 - vectorbt.utils.schedule - INFO - Starting schedule manager with jobs [Every 10 seconds do update() (last run: [never], next run: 2021-03-22 15:20:24)]
2021-03-22 15:20:26,670 - vectorbt.data.updater - INFO - Updated data has 60 rows from 2021-03-22 13:21:00+00:00 to 2021-03-22 14:20:00+00:00
2021-03-22 15:20:26,868 - numexpr.utils - INFO - NumExpr defaulting to 4 threads.
2021-03-22 15:20:38,956 - vectorbt.data.updater - INFO - Updated data has 60 rows from 2021-03-22 13:21:00+00:00 to 2021-03-22 14:20:00+00:00
2021-03-22 15:20:50,839 - vectorbt.data.updater - INFO - Updated data has 60 rows from 2021-03-22 13:21:00+00:00 to 2021-03-22 14:20:00+00:00
2021-03-22 15:21:03,031 - vectorbt.data.updater - INFO - Updated data has 61 rows from 2021-03-22 13:21:00+00:00 to 2021-03-22 14:21:00+00:00
2021-03-22 15:21:15,360 - vectorbt.data.updater - INFO - Updated data has 61 rows from 2021-03-22 13:21:00+00:00 to 2021-03-22 14:21:00+00:00
2021-03-22 15:21:27,299 - vecto

In [10]:
telegram_bot.stop()

2021-03-22 15:22:39,661 - vectorbt.utils.messaging - INFO - Stopping bot
2021-03-22 15:22:39,664 - apscheduler.scheduler - INFO - Scheduler has been shut down
